## 신경망 모델 구성하기
torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. <br>
PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

In [ ]:
# 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다.
# torch.cuda 또는 torch.backends.mps 가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용합니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### 클래스 정의하기

In [ ]:
#📌 신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화합니다.
#📌 nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
#📌 NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력합니다.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)  # 배치크기 1, 이미지의 크기 28x28
logits = model(X)  # X를 모델에 입력, logits는 각 클래스에 대한 점수를 의미
pred_probab = nn.Softmax(dim=1)(logits)  # 소프트맥스 적용해 클래스별 확률 분포로 변환
y_pred = pred_probab.argmax(1)  # 가장 확률이 높은 클래스를 예측값으로 선택
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


---

### 모델 계층 (Layer)
28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [ ]:
#📌 nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [11]:
#📌 Linear 계층은 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈임
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [12]:
#📌 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다. 
#📌 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.2720,  0.2158,  0.7118, -0.0889,  0.1320,  0.0789, -0.1721, -0.1163,
          0.7624,  0.0684, -0.9619, -0.0410, -0.0498,  0.0456, -0.0387,  0.1102,
         -0.4683,  0.0362, -0.0891,  0.2415],
        [-0.0863,  0.4445,  0.3661, -0.2910,  0.0018,  0.0384,  0.4135,  0.1175,
          0.4792, -0.0431, -0.5644,  0.1001,  0.1270, -0.4057,  0.5302,  0.2814,
         -0.6950, -0.0415, -0.1329, -0.0043],
        [-0.1910,  0.2873,  0.5827, -0.4594, -0.2557, -0.2483,  0.2404, -0.0531,
          0.4618, -0.0686, -0.6308,  0.2566, -0.1816, -0.1990,  0.1739,  0.3518,
         -0.3067, -0.3344, -0.2308,  0.6522]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2158, 0.7118, 0.0000, 0.1320, 0.0789, 0.0000, 0.0000, 0.7624,
         0.0684, 0.0000, 0.0000, 0.0000, 0.0456, 0.0000, 0.1102, 0.0000, 0.0362,
         0.0000, 0.2415],
        [0.0000, 0.4445, 0.3661, 0.0000, 0.0018, 0.0384, 0.4135, 0.1175, 0.4792,
         0.0000, 0.0000, 0.1001, 0.1270, 0.0000, 0.53

### nn.Sequential

In [13]:
#📌 nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 
#📌 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

In [ ]:
#📌 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환하는데,
#📌 이 logits는 Softmax를 거치며 각 클래스에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정된다.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)